In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022487640380859375
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 19.358263073186276
200000 33.259586320395144
300000 34.445046350393625
400000 33.61768324238777
500000 34.35697408344483
600000 32.648381717723396
700000 31.891078136535686
800000 32.427549383035306
900000 26.96771553174566
1000000 18.476400309050145
1100000 31.30972903912864
1200000 31.318877735099555
1300000 31.22738679408435
1400000 32.85780583275484
1500000 31.93586281491381
1600000 31.734249210966784
1700000 32.169014836280105
1800000 29.889310117742905
1900000 14.223196386198723
2000000 31.402537849369022
2100000 30.921154881445585
2200000 30.583665241604095
2300000 32.46846765595523
2400000 30.79841271232487
2500000 30.59271183570289
2600000 30.351468298801542
2700000 29.520995255147703
2800000 13.205324262654164
2900000 29.906957823216608
3000000

29500000 31.081674049298417
29600000 29.092459101380136
29700000 28.918896700891853
29800000 26.898518631426892
29900000 26.383629963141573
30000000 12.109063871904826
30100000 26.764965124917442
30200000 25.864241525069495
30300000 28.41496666752278
30400000 28.741099305656565
30500000 30.292081864958373
30600000 29.03177758786132
30700000 15.226986552960824
30800000 29.47250266934066
30900000 29.422737421289504
31000000 31.242771362292306
31100000 29.911341293467284
31200000 15.598873476734202
31300000 31.153914323548396
31400000 29.923515149453134
31500000 30.422989488060484
31600000 28.54613717057667
31700000 27.32265448678719
31800000 15.44435809460982
31900000 23.381915685558162
32000000 27.962592567829606
32100000 29.114075152780327
32200000 29.256921940343663
32300000 30.61416201703489
32400000 32.057016223299804
32500000 15.739440356196393
32600000 30.597511418426702
32700000 31.064713199997676
32800000 28.995802359507685
32900000 29.51464179571382
33000000 15.130433587005333


59300000 12.361951731033502
59400000 13.902255157854118
59500000 10.418894472891328
59600000 17.057245388069997
59700000 21.122529112739656
59800000 13.209296359282677
59900000 12.954774763019968
60000000 20.960635316192903
60100000 15.024835836578982
60200000 14.606086267689227
60300000 14.399755904696304
60400000 12.640357373464592
60500000 9.818512907343269
60600000 15.384090036637629
60700000 12.61143916852426
60800000 18.314621997802806
60900000 4.967456385840247
61000000 16.352558231801822
61100000 12.374658506600975
61200000 19.128412050211047
61300000 8.261520663951325
61400000 14.378785624085983
61500000 13.933716606790483
61600000 16.63348573166418
61700000 13.44154538093389
61800000 13.525398191871158
61900000 16.8444273294906
62000000 12.876014978748799
62100000 16.170596671313184
62200000 14.437736087429087
62300000 5.930870669250876
62400000 11.33132856888618
62500000 11.427309147286937
62600000 13.854144228634786
62700000 4.900242308388346
62800000 12.789832968224735
629

89000000 26.553421241288543
89100000 30.040062407485937
89200000 24.755539207236147
89300000 24.731178901645197
89400000 24.33989549026583
89500000 25.036984122208604
89600000 24.234881370396792
89700000 19.894912876110695
89800000 15.192553638835884
89900000 14.66898062595293
90000000 17.982029766228816
90100000 12.957172935798566
90200000 24.607550250266247
90300000 23.935388792188135
90400000 24.655785780687093
90500000 24.731214591420162
90600000 24.040493907165644
90700000 27.650734336684067
90800000 27.988813891022
90900000 27.09357307349474
91000000 24.860177179032593
91100000 23.830780961758226
91200000 23.478372860972915
91300000 24.4761627066394
91400000 23.16362347154101
91500000 24.08979664381431
91600000 11.512625069878903
91700000 17.793813947450786
91800000 15.620606235725624
91900000 13.43559746046603
92000000 22.878279346374214
92100000 23.68893302402656
92200000 25.2448639670557
92300000 22.734883167159357
92400000 22.738228218945462
92500000 25.08768646518371
9260000

118100000 28.109624491318822
118200000 25.455586878093182
118300000 24.246265006428203
118400000 23.722675282513528
118500000 23.84113166220851
118600000 25.164688463984263
118700000 23.504155924544605
118800000 23.573610463730887
118900000 15.863415690377119
119000000 3.6084128422673145
119100000 16.200005619616018
119200000 24.107147849989907
119300000 24.077101816983372
119400000 23.955590063618097
119500000 24.785557366199313
119600000 22.996267619645597
119700000 25.19590449302756
119800000 25.05770341201928
119900000 25.30038775477515
120000000 20.984673532565875
120100000 20.025301036237483
120200000 19.030861121383648
120300000 20.67294175248722
120400000 21.469360772573264
120500000 19.530742603799073
120600000 19.471283207423184
120700000 14.321367920678854
120800000 3.903901827045888
120900000 11.155907270452573
121000000 20.128424463861535
121100000 19.689662936270754
121200000 20.040841593962952
121300000 20.581241507046336
121400000 19.608977327658653
121500000 20.0844586

...

319900000 4.347595345773574
320000000 6.444524180700153
320100000 5.833051171466466
320200000 5.499621921541373
320300000 5.885815112596482
320400000 5.671124073189457
320500000 2.3879652107870446
320600000 2.834761204918474
320700000 5.911759134743734
320800000 6.021848967990269
320900000 6.541488035792559
321000000 6.410290623651104
321100000 6.454341929038495
321200000 6.537737076645792
321300000 6.053229142014643
321400000 6.677254250590225
321500000 6.180819636623104
321600000 2.9822507378446823
321700000 5.0366946694660255
321800000 5.896383378192544
321900000 5.882656016288671
322000000 5.870055694094257
322100000 6.042247955166578
322200000 5.436783431952874
322300000 3.5375540137282706
322400000 1.7993298881656923
322500000 5.906270452789466
322600000 5.975030539195326
322700000 6.2106494755634865
322800000 6.21681917723446
322900000 6.510797127821496
323000000 6.5075085933779775
323100000 6.12194782606562
323200000 6.72561245254013
323300000 6.4930722837553025
323400000 2.932

348200000 2.8236927702249885
348300000 3.057719342879441
348400000 3.622530829792357
348500000 3.0365144345801474
348600000 3.983021225804799
348700000 3.8761144625816013
348800000 4.3192912337589755
348900000 3.190978111345116
349000000 3.5496038196920416
349100000 3.1534284311977925
349200000 2.775031867282205
349300000 3.6232988774731685
349400000 2.6550917349805734
349500000 4.036991589984927
349600000 3.8686538525278036
349700000 3.5617108205902674
349800000 2.970469616109732
349900000 3.447765652943929
350000000 2.7680477289700804
350100000 2.46967035839814
350200000 3.227474884323087
350300000 2.6394233859191836
350400000 3.4817290782239305
350500000 3.3945397545072136
350600000 3.4545929877454133
350700000 2.799878302478721
350800000 3.553033577962772
350900000 3.5134263468601112
351000000 1.5510554480002192
351100000 3.2619575458444654
351200000 2.110207354365776
351300000 3.4470899448678956
351400000 3.5697162994672063
351500000 2.7481052851691348
351600000 3.064529001711064


376700000 1.501344950590064
376800000 1.5327410645181072
376900000 1.52693871360498
377000000 1.3783385271413326
377100000 1.203540125126683
377200000 2.0425462728692825
377300000 1.1889611158066988
377400000 2.386812546465867
377500000 1.1939063909172547
377600000 1.3408258554702301
377700000 1.4902870647017659
377800000 1.4352677683545914
377900000 1.7141013607041538
378000000 1.0236193278795447
378100000 1.3776481379507581
378200000 0.8059096142719587
378300000 1.3710629936192598
378400000 1.948865373840915
378500000 1.6835310801406405
378600000 1.9830705019987658
378700000 1.9796079097967048
378800000 1.838941635914773
378900000 0.9782949177468784
379000000 1.361539899983085
379100000 1.0605636260708768
379200000 1.6044494788588914
379300000 1.407586260433005
379400000 1.46507526685973
379500000 1.7100952263095777
379600000 1.538783794089251
379700000 1.583863104052037
379800000 1.3033285080183843
379900000 1.4076741562698714
380000000 1.2672032231039505
380100000 1.31883652070471


404000000 0.444926983657211
404100000 0.8246753466854426
404200000 0.70533174823708
404300000 0.5791559854762951
404400000 0.45714688088025324
404500000 0.6858911097479263
404600000 0.7334372901292044
404700000 0.6518024879568484
404800000 0.48543175362011787
404900000 0.39561652400940656
405000000 0.6293457364387082
405100000 0.7016070886432662
405200000 0.6622138978628118
405300000 0.3110485242246555
405400000 0.5787150018860929
405500000 0.663568569604526
405600000 0.6298543518535594
405700000 0.6100774359469812
405800000 0.3326748608174026
405900000 0.5922199208231304
406000000 1.0457987064937915
406100000 0.5942508038021823
406200000 0.3041030302783284
406300000 0.5789683382977976
406400000 0.6239371589439908
406500000 0.7264672954489323
406600000 0.6403946702621182
406700000 0.3793804307041168
406800000 0.6744774569457835
406900000 0.4509898027463456
407000000 0.1141355220533212
407100000 0.1106773436557154
407200000 0.11181759622116222
407300000 0.11380226677816112
407400000 0.1

430300000 0.09667323512379329
430400000 0.06509629653340578
430500000 0.0935079841871692
430600000 0.06749795216760039
430700000 0.08261428612458706
430800000 0.08040892694310348
430900000 0.0652247639744911
431000000 0.08552864561221003
431100000 0.058912867723491455
431200000 0.08267216603402959
431300000 0.05772612833450237
431400000 0.07825572068248192
431500000 0.060362596678041745
431600000 0.07166255180481076
431700000 0.07158413392050399
431800000 0.054810375245683725
431900000 0.07103970399829745
432000000 0.04976367578825686
432100000 0.07209907309273877
432200000 0.05084975128399332
432300000 0.06241439437933763
432400000 0.05435505999564132
432500000 0.058881419312099614
432600000 0.05905765945218007
432700000 0.045313531565298645
432800000 0.06324849192130566
432900000 0.04630147560929258
433000000 0.05172250062288509
433100000 0.039426197499324876
433200000 0.04599943007147975
433300000 0.044289858962251086
433400000 0.03724386366848151
433500000 0.043631166353434316
4336